<a href="https://colab.research.google.com/github/lucdekeijzer/film_recommendations/blob/main/chatbot_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install colorama
!pip install scikit_learn
!pip install Flask
!pip install beautifulsoup4
!pip install bs4
!pip install googletrans==4.0.0rc1

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 11.2 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=226cdd24ee61f8d7038132303d3b755f6e3bce39967a7b02169c520ceb538404
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [3]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import googletrans
from googletrans import Translator
import requests
import pandas as pd


In [4]:
#with open('intents.json') as file:
    #data = json.load(file)

data = pd.read_json("https://raw.githubusercontent.com/lucdekeijzer/film_recommendations/main/intents.json")
 

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 10)                170       
                                                                 
Total params: 16,714
Trainable params: 16,714
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
3/3 [==============================] - 1s 6ms/step - loss: 2.2978 - accuracy: 0.1562
Epoch 2/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2937 - accuracy: 0.1562
Epoch 3/500
3/3 [==============================] - 0s 7ms/step - loss: 2.2908 - accuracy: 0.1562
Epoch 4/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2872 - accuracy: 0.1562
Epoch 5/500
3/3 [==============================] - 0s 6ms/step - loss: 2.2835 - accuracy: 0.1562
Epoch 6/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2797 - accuracy: 0.1562
Epoch 7/500
3/3 [==============================] - 0s 5ms/step - loss: 2.2763 - accuracy: 0.1562
Epoch 8/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2724 - accuracy: 0.1562
Epoch 9/500
3/3 [==============================] - 0s 5ms/step - loss: 2.2685 - accuracy: 0.1562
Epoch 10/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2645 - accuracy: 0.1562
Epoch 11/500
3/3 [===========

In [9]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [10]:
def webscraper(x):
  from urllib.request import Request, urlopen
  from bs4 import BeautifulSoup
  import requests
  from random import randint

  #x = inp

  root = "https://www.google.com"
  link = "https://www.google.com/search?q=top+10+{}+movies&oq=top+&aqs=chrome.0.69i59l3j69i57j69i60l4.651j0j4&sourceid=chrome&ie=UTF-8".format(x)
  results = []

  req = Request(link, headers = {"User-Agent": "Mozilla/5.0"})
  webpage = urlopen(req).read()
  with requests.Session() as c:
    soup = BeautifulSoup(webpage, "html5lib")
    #print(soup)
    for item in soup.find_all('div', attrs = {"class": "RWuggc"}):
      results.append(item)

  titles = []
  
  for word in results:
    for j in word:
      for i in j:
        for p in i:
          is_year_of_pub = True                          #filtering out years of publication; filters everything that consists only of numbers, so a movie like 2012 will not make it to the recommendation
          for char in p:
            if ord(char) < 48 or ord(char) > 57:
              is_year_of_pub = False
              break
          if is_year_of_pub == False:                     #Only if p is not a string of just numbers, it is put in the titles list
            titles.append(p)

  #print(titles)
  end_of_titles = len(titles)
  movie_index = randint(0, end_of_titles )
  movie_index_end = movie_index + 1
  return str(*(titles[movie_index:movie_index_end]))


 

In [19]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle
translator = Translator()

#with open("intents.json") as file:
#    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    language_detect = input("Please state the language you would like to talk in to Moviebot? \n (State the language in English please) ")
    language_detect = language_detect.lower()
    language_lib = googletrans.LANGCODES

    
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        #language = (translator.detect(inp)).lang
        language = language_lib[language_detect]
        inp_translated = (translator.translate(inp)).text
        if inp_translated.lower() == "quit":
            break
        


        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp_translated]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
          if i['tag'] == tag:
            answer = np.random.choice(i['responses'])
            trans_answer = translator.translate(answer, dest = language).text
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)

        if tag == "thanks":
            answer = np.random.choice(i['responses'])
            trans_answer = translator.translate(answer, dest = language).text
            #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
            break
        if tag == "genres":
          answer = np.random.choice(i['responses'])
          trans_answer = translator.translate(answer, dest = language).text
          #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
          print(translator.translate("What about: ", dest = language).text, webscraper(inp_translated))
          genre = inp_translated
        if tag == "another":
          answer = np.random.choice(i['responses'])
          trans_answer = translator.translate(answer, dest = language).text
          #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
          print(webscraper(genre))
        
          
          



        #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
Please state the language you would like to talk in to Moviebot? 
 (State the language in English please) nederlands
User: hallo


KeyError: ignored

In [29]:
#print(googletrans.LANGUAGES)

example = "Hallo, hoe gaat het?"
transl_ex = (translator.translate(example)).text
print(transl_ex)


language = (translator.detect("Lorem Ipsum")).lang
print(language)

print((translator.translate(transl_ex, dest = language)).text)

#translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
#for translation in translations:
#print(translation.origin, ' -> ', translation.text)
# The quick brown fox  ->  빠른 갈색 여우
# jumps over  ->  이상 점프
# the lazy dog  ->  게으른 개

Hello how are you?
la
Salve, quam vos?


In [ ]:
titles = []
for word in results:
  for j in word:
    for i in j:
      for p in i:
        titles.append(p)

print(titles)


In [ ]:
#googletrans.LANGCODES

#print("In what language would you like to talk to Moviebot?")
language_detect = input("In what language would you like to talk to Moviebot? ")
language_detect = language_detect.lower()
language_lib = googletrans.LANGCODES

print(language_lib[language_detect])

In what language would you like to talk to Moviebot? Dutch
nl


In [36]:
#Testing
#1. Unit Testing
def LangDetectTest():
  correct_input = "Lorem Ipsum"
  incorrect_input = "fraeovknls vnaeilvk"

  test1 = (translator.detect(correct_input)).lang
  
  try:
    test2 = (translator.detect(incorrect_input)).lang
  except:
    test2 = "Error"
    return "Result of language detector test with correct input: {}; desired output: la \n\
Result of language detector test with incorrect input was an {}".format(test1, test2)
  else:
    return "Result of language detector test with correct input: {}; desired output: la \n\
Result of language detector test with incorrect input: {}".format(test1, test2)


#2. Component/Integration testing

def WebscraperTest():
  known_genre = "action"
  unknown_genre = "bruh"

  test1 = webscraper(known_genre)
  try:
    test2 = webscraper(unknown_genre)
  except:
    test2 = "Error"
    return "Result of webscraper test with a known genre as input: {}\n\
    Result of webscraper test with an unknown genre as input was an {}".format(test1, test2)
  else:
    return "Result of webscraper test with a known genre as input: {}\n\
Result of webscraper test with an unknown genre as input: {}"\
  .format(test1, test2)


print(LangDetectTest())
print(WebscraperTest())


Result of language detector test with correct input: la; desired output: la 
Result of language detector test with incorrect input: en
Result of webscraper test with a known genre as input: Free Guy
Result of webscraper test with an unknown genre as input: 
